<a href="https://colab.research.google.com/github/Neel7317/Machine_learning/blob/master/Post_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
#putting movies data on 'movies' dataframe


In [24]:
df = pd.read_csv('posts.csv')#read data from CSV file

In [25]:
df.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [26]:
df = df.assign(_id=(df['title'] + '_' + df['category']).astype('category').cat.codes)#make id int from hexa decimal
df.head()

,_id,title,category,post_type
0,443,hello there,Plant Biotechnology,blog
1,248,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,418,What is an Operating System ?,Operating Systems,blog
3,227,Lord Shiva,Drawings,artwork
4,177,How Competition law evolved?,Competition Laws,blog


In [27]:
df['category'][0]

'Plant Biotechnology'

In [28]:
tfidf = TfidfVectorizer(stop_words='english')

In [29]:
df['category'] = df['category'].fillna('')#fill NaN value with blank space string

In [30]:
#Construct the required TF-IDF matrix by applying the fit_transform method on the category feature
category_matrix = tfidf.fit_transform(df['category'])

In [31]:
##Output the shape of tfidf_matrix
category_matrix.shape

(493, 266)

In [32]:
similarity_matrix = linear_kernel(category_matrix,category_matrix)

similarity_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.15033338,
        0.30692944],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.15033338, 0.        , ..., 0.        , 1.        ,
        0.31439084],
       [0.        , 0.30692944, 0.        , ..., 0.        , 0.31439084,
        1.        ]])

In [33]:
#posts index mapping
mapping = pd.Series(df.index,index = df['title'])
mapping

title
hello there                                                              0
Ml and AI                                                                1
What is an Operating System ?                                            2
Lord Shiva                                                               3
How Competition law evolved?                                             4
                                                                      ... 
CI/CD using GitHub Actions                                             488
Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch     489
Recommendation Engine                                                  490
Face Recognition using Transfer Learning.                              491
Configure CI/CD Pipeline in GitLab and deployment to server via SSH    492
Length: 493, dtype: int64

In [36]:
def recommend_movies_based_on_plot(title):
  post_index= mapping[title]
  #get similarity values with other movies
  #similarity_score is the list of index and similarity matrix
  similarity_score = list(enumerate(similarity_matrix[post_index]))
  #sort in descending order the similarity score of movie inputted with all the other movies
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
  # Get the scores of the 15 most similar movies. Ignore the first movie.
  similarity_score = similarity_score[1:10]
  #return movie names using the mapping series
  movie_indices = [i[0] for i in similarity_score]
  return (df['title'].iloc[movie_indices])

In [43]:
print(recommend_movies_based_on_plot('Ml and AI'))#input post title and recommend post similar to that post

122                              Artificial Intelligence
170                                         Types Of AI.
186    7 Best Python Data Science Courses & Certifica...
224    10 Best Artificial Intelligence (AI) Courses O...
450               Machine Learning”&“Operations" (MlOps)
454                                   GAN's INTRODUCTION
455                                        GAN's Part(2)
456                   Recommend Systems Machine Learning
486                                          Learning...
Name: title, dtype: object


In [44]:
print(recommend_movies_based_on_plot('Artificial Intelligence'))#input post title and recommend post similar to that post

170                                         Types Of AI.
169             What sports will look like in the future
1                                              Ml and AI
291             8 Best Machine Learning Courses for 2020
375    6 Best + Free PLC Programming Training & Cours...
292    10 Best UI UX Design Course & Certification [2...
106                                Topology & it's types
107                                 Baseband & Broadband
108                                   Information System
Name: title, dtype: object


In [45]:
print(recommend_movies_based_on_plot('Lord Shiva'))

11          Trident
12       Lord shiva
15           Kid Bu
26    Shadow Sketch
30       The Chosen
34          Spartan
45     ROMAN REIGNS
46               OM
47        MK Swords
Name: title, dtype: object
